In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import gc

In [2]:


# Define the folder where files are stored
data_folder = '/home/amy/projects/Tigo/data/unzipped_data/'

# List all files in the folder (you can filter for specific extensions if needed)
all_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.csv')]

# Initialize an empty list to store dataframes
dataframes = []

# Process each file
for file_path in all_files:
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Process each line
    data = []
    for line in lines:
        # Split by ';' first
        parts = line.strip().split(';')
        # For the last part, split by ','
        last_part = parts[-1].split(',')
        # Combine parts
        data.append(parts[:-1] + last_part)

    # Create a DataFrame for the current file
    df = pd.DataFrame(data, columns=['NO_FACTURA',
                                     'NOMBRE_EMISOR',
                                     'MONEDA',
                                     'FECHA_EMISION',
                                     'FECHA_VENCIMIENTO',
                                     'MONTO_FACTURA',
                                     'FECHA_PAGO',
                                     'MONTO_PAGO',
                                     'PERIODO'])  # Adjust column names
    
    # Set the first row as the header and drop it from the data
    df.columns = df.iloc[0]  # Promote the first row to header
    df = df[1:]  # Drop the first row from data
    df.reset_index(drop=True, inplace=True)  # Reset the index
    
    # Add the file name as a new column (optional, for traceability)
    df['Source_File'] = os.path.basename(file_path)
    
    # Append the processed DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one
final_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print(final_df.head())


0 NO_FACTURA NOMBRE_EMISOR MONEDA FECHA_EMISION   FECHA_VENCIMIENTO  \
0   99790428          TIGO    COP      20231119  05-DIC-23 00:00:00   
1   99744447          TIGO    COP      20231116  04-DIC-23 00:00:00   
2   68299551          TIGO    COP      20231110  28-NOV-23 00:00:00   
3   19065897          TIGO    COP      20231107  28-NOV-23 00:00:00   
4   29214432          TIGO    COP      20231113  01-DIC-23 00:00:00   

0 MONTO_FACTURA          FECHA_PAGO MONTO_PAGO PERIODO  \
0          1544  22-NOV-23 00:00:00                 39   
1           161  24-AGO-24 00:00:00        160      85   
2         31936  30-NOV-23 00:00:00      31936    None   
3         35910  01-DIC-23 00:00:00      35910      07   
4         37425  20-NOV-23 00:00:00      37425    None   

0                      Source_File  
0  Compra_Cartera_202311_Part3.csv  
1  Compra_Cartera_202311_Part3.csv  
2  Compra_Cartera_202311_Part3.csv  
3  Compra_Cartera_202311_Part3.csv  
4  Compra_Cartera_202311_Part3.csv  


In [3]:
gc.enable()
gc.collect()

0

In [4]:
final_df.drop(columns='NO_FACTURA', inplace=True)
final_df.drop(columns='PERIODO', inplace=True)

In [5]:
final_df['FECHA_EMISION'] = pd.to_datetime(final_df['FECHA_EMISION'], format='%Y%m%d', errors='coerce').dt.strftime('%Y-%m-%d')

In [6]:
gc.collect()

18

In [8]:
final_df.drop(columns='Source_File', inplace=True)

KeyError: "['Source_File'] not found in axis"

In [9]:
# Map Spanish months to English
month_map = {
    'ENE': 'JAN', 'FEB': 'FEB', 'MAR': 'MAR', 'ABR': 'APR',
    'MAY': 'MAY', 'JUN': 'JUN', 'JUL': 'JUL', 'AGO': 'AUG',
    'SEP': 'SEP', 'OCT': 'OCT', 'NOV': 'NOV', 'DIC': 'DEC'
}


# Apply mapping only to non-null values
def map_months(date):
    if pd.isnull(date) or date == '':
        return date  # Leave NaN or empty values as is
    for sp_month, en_month in month_map.items():
        date = date.replace(sp_month, en_month)  # Replace Spanish month with English
    return date

# Apply the function to the column
final_df['FECHA_VENCIMIENTO'] = final_df['FECHA_VENCIMIENTO'].apply(map_months)
final_df['FECHA_PAGO'] = final_df['FECHA_PAGO'].apply(map_months)

In [10]:
# Convert the column with the date format '27-SEP-23 00:00:00' to '%Y-%m-%d'
final_df['FECHA_VENCIMIENTO'] = pd.to_datetime(final_df['FECHA_VENCIMIENTO'], format='%d-%b-%y %H:%M:%S', errors='coerce').dt.strftime('%Y-%m-%d')

# Convert the column with the date format '27-SEP-23 00:00:00' to '%Y-%m-%d'
final_df['FECHA_PAGO'] = pd.to_datetime(final_df['FECHA_PAGO'], format='%d-%b-%y %H:%M:%S', errors='coerce').dt.strftime('%Y-%m-%d')

In [11]:
final_df['FECHA_PAGO'] = pd.to_datetime(final_df['FECHA_PAGO'], format='%Y-%m-%d', errors='coerce')
final_df['FECHA_VENCIMIENTO'] = pd.to_datetime(final_df['FECHA_VENCIMIENTO'], format='%Y-%m-%d', errors='coerce')

final_df['days_to_pay'] = (final_df['FECHA_PAGO'] - final_df['FECHA_VENCIMIENTO']).dt.days

In [12]:
gc.collect()

594

In [13]:
# Replace empty strings with NaN
# Remove everything after the comma (including the comma)
final_df['MONTO_PAGO'] = final_df['MONTO_PAGO'].str.split(',').str[0]
final_df['MONTO_PAGO'] =final_df['MONTO_PAGO'].replace('', np.nan)
final_df['MONTO_PAGO'] = pd.to_numeric(final_df['MONTO_PAGO'] , errors='coerce')
final_df['MONTO_PAGO'].fillna(0)

0               0.0
1             160.0
2           31936.0
3           35910.0
4           37425.0
             ...   
53071838    49900.0
53071839    28815.0
53071840        0.0
53071841    17034.0
53071842    31920.0
Name: MONTO_PAGO, Length: 53071843, dtype: float64

In [14]:
gc.collect()

14

In [15]:
final_df['MONTO_FACTURA'] = final_df['MONTO_FACTURA'].replace('', np.nan)



# Remove everything after the comma (including the comma)
final_df['MONTO_FACTURA'] = final_df['MONTO_FACTURA'].str.split(',').str[0]


# Convert to float
final_df['MONTO_FACTURA'] = pd.to_numeric(final_df['MONTO_FACTURA'] , errors='coerce')
final_df['MONTO_FACTURA'] = final_df['MONTO_FACTURA'].astype('float')


In [16]:
gc.collect()

18

In [17]:
final_df['monetary_loss'] = (final_df['MONTO_FACTURA'] - final_df['MONTO_PAGO'])
# Where MONTO_PAGO is NaN, set monetary_loss to MONTO_FACTURA
final_df['monetary_loss'] = final_df['monetary_loss'].fillna(final_df['MONTO_FACTURA'])

In [18]:
gc.collect()

0

In [19]:
final_df['loss_percentage'] = ((final_df['MONTO_FACTURA'] - final_df['MONTO_PAGO'])/final_df['MONTO_FACTURA']).fillna(1)

In [ ]:
final_df.to_csv('/home/amy/projects/Tigo/data/cleaned_data.csv')